In [2]:
import pandas as pd
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel
csv_file_path = "1_million_cleaned_data.csv"  # Specify your dataset file
df = pd.read_csv(csv_file_path)
df=df[:10]
df

,Note_ID,SentID,SentText
0,850646257,18846892,if you need lifting assistance getting out of ...
1,1441026653,18846892,impression: 1. polyps x 10 as above 2. pandive...
2,515741265,18846893,colon: postsurgical changes of right hemicolec...
3,53588620,18846893,results review from this visit results for ord...
4,1058913495,18846892,: 2 pain located in your lower body such as yo...
5,1438711694,18846891,medical oncology clinic note: here to get eval...
6,968582122,18846891,bed: 06 expected date: expected time: means of...
7,31338873,18846893,he has no other complaints. objective: bp 143/...
8,1193388192,18846893,stable trace ascites in the deep pelvis - 7/30...
9,1045567498,18846891,clinic note - medicare visit encounter date: 2...


In [30]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, GPT2Config, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from torch.utils.data import Dataset
import pandas as pd
import torch
torch.cuda.empty_cache()


from transformers import AutoTokenizer, GPT2LMHeadModel, GPT2Config, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from torch.utils.data import Dataset
import pandas as pd

class CustomTextDataset(Dataset):
    def __init__(self, tokenizer, texts, max_length):
        self.tokenizer = tokenizer
        self.texts = texts
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        encoded_input = self.tokenizer(
            self.texts[idx],
            truncation=True,
            max_length=self.max_length
        )
        return {"input_ids": encoded_input["input_ids"]}

def fine_tune_gpt2(df, model_name="microsoft/BioGPT", output_dir="./symptom-BioGPT-1 Million"):
    if 'SentText' not in df.columns:
        raise ValueError("DataFrame must contain a 'SentText' column.")

    texts = df['SentText'].tolist()

    # Load tokenizer and model with the correct configuration
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    config = GPT2Config.from_pretrained(model_name)
    model = GPT2LMHeadModel.from_pretrained(model_name, config=config)

    # Ensure padding token is set if missing
    if tokenizer.pad_token is None:
        tokenizer.add_special_tokens({'pad_token': tokenizer.eos_token})
        model.resize_token_embeddings(len(tokenizer))

    # Prepare custom dataset
    dataset = CustomTextDataset(tokenizer, texts, max_length=512)
    data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
    
    # Define training arguments with reduced batch size and enabled FP16
    training_args = TrainingArguments(
        output_dir=output_dir,
        overwrite_output_dir=True,
        num_train_epochs=4,
        per_device_train_batch_size=2,  # Reduced batch size
        gradient_accumulation_steps=256,
        learning_rate=5e-5,
        weight_decay=0.01,
        save_steps=10_000,
        save_total_limit=2,
        fp16=True,  # Enable FP16 to reduce memory usage
    )

    # Initialize Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=data_collator,
        train_dataset=dataset,
    )

    # Start training
    trainer.train()

    # Save model and tokenizer
    model.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)

# Ensure df is defined before calling this function
# Example: df = pd.DataFrame({'SentText': ["example sentence 1", "example sentence 2"]})
 fine_tune_gpt2(df)


Using pad_token, but it is not set yet.
/home/nzeinali/.local/lib/python3.8/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
